In [ ]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, cohen_kappa_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from scipy.signal import resample
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
from sklearn.utils.class_weight import compute_class_weight
import seaborn as sns
import matplotlib.pyplot as plt

# Load datasets
eeg_data = pd.read_csv('eeg_data_trimmed.csv')
emg_data = pd.read_csv('emg_data_trimmed.csv')
labels = pd.read_csv('labels_trimmed.csv')

# Label encoding
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels.values.ravel())

# Check for class imbalance
class_weights = compute_class_weight('balanced', classes=np.unique(labels), y=labels)
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Models to evaluate
models = {
    "SVM": SVC(kernel='linear', C=0.0001, class_weight=class_weights_dict),
    "Random Forest": RandomForestClassifier(n_estimators=5, max_depth=5, random_state=42, class_weight='balanced'),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "LDA": LinearDiscriminantAnalysis(),
    "Naive Bayes": GaussianNB()
}

# Stratified k-Fold Cross-Validation
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold_results = {model_name: [] for model_name in models.keys()}

for fold, (train_index, test_index) in enumerate(kf.split(eeg_data, labels)):
    print(f"\nFold {fold + 1}/5")
    eeg_train, eeg_test = eeg_data.iloc[train_index].values, eeg_data.iloc[test_index].values
    emg_train, emg_test = emg_data.iloc[train_index].values, emg_data.iloc[test_index].values
    y_train, y_test = labels[train_index], labels[test_index]

    # Resample EEG and EMG
    eeg_train = resample(eeg_train, num=len(emg_train), axis=0)
    eeg_test = resample(eeg_test, num=len(emg_test), axis=0)

    # Align using DTW
    distance, path = fastdtw(eeg_train, emg_train, dist=euclidean)
    aligned_eeg_train = eeg_train[[p[0] for p in path]]
    aligned_emg_train = emg_train[[p[1] for p in path]]
    distance, path = fastdtw(eeg_test, emg_test, dist=euclidean)
    aligned_eeg_test = eeg_test[[p[0] for p in path]]
    aligned_emg_test = emg_test[[p[1] for p in path]]

    # Combine and scale
    X_train = np.hstack((aligned_eeg_train, aligned_emg_train))
    X_test = np.hstack((aligned_eeg_test, aligned_emg_test))
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Train and evaluate each model
    for model_name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        precision = precision_score(y_test, y_pred, average='weighted')
        recall = recall_score(y_test, y_pred, average='weighted')
        f1 = f1_score(y_test, y_pred, average='weighted')
        accuracy = accuracy_score(y_test, y_pred)
        kappa = cohen_kappa_score(y_test, y_pred)
        
        # Store results
        fold_results[model_name].append({"precision": precision, "recall": recall, "f1": f1, "accuracy": accuracy, "kappa": kappa})
        
        # Print per-fold results
        print(f"{model_name} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}, Kappa: {kappa:.4f}")
        
        # Confusion matrix visualization
        cm = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(6, 4))
        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix - {model_name} (Fold {fold+1})")
        plt.show()

# Online Adaptation Function
def online_adaptation_with_regularization(model, optimizer, buffer_X, buffer_y, criterion, val_loader, num_cycles=3, batch_size=16, lr=0.0001):
    model.train()
    buffer_dataset = EEGEMGDataset(buffer_X, buffer_y)
    buffer_loader = DataLoader(buffer_dataset, batch_size=batch_size, shuffle=True)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=0.005)
    
    for cycle in range(num_cycles):
        print(f"Online Adaptation Cycle {cycle + 1}/{num_cycles}")
        for X_batch, y_batch in buffer_loader:
            eeg = X_batch[:, :input_dim].unsqueeze(1)
            emg = X_batch[:, input_dim:].unsqueeze(1)
            optimizer.zero_grad()
            outputs = model(eeg, emg)
            loss = criterion(outputs, y_batch)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
        
        # Evaluate on validation set
        print("Validation Metrics After Cycle:")
        evaluate_model(model, val_loader)

# Average metrics across folds
print("\nAverage Metrics Across Folds:")
for model_name, results in fold_results.items():
    avg_metrics = {metric: np.mean([res[metric] for res in results]) for metric in results[0]}
    print(f"\nModel: {model_name}")
    for metric, value in avg_metrics.items():
        print(f"{metric.capitalize()}: {value:.4f}")
